# 作業：基因演算法

- 程式檔
- 影片檔（文字詳細說明也可）
- 兩張圖
    - log: Gen, best fit, x, y, x+y, history_best_fit
    - 圖表：Fitness/lteration

## 題目

1. **目標函數**：

   $$\text{Minimize } f(x, y) = -\cos(\pi y) - \exp(-\pi (x - 0.5)^2) \sin^2(\pi x)$$

2. **約束條件**：

   $$-1 \leq x \leq 2$$
   $$4 \leq y \leq 6$$
   $$x + y \geq 5$$


2. **答案**：

   $$\text{Minimum } = -2 \text{ at } (x, y) = (0.5, 6)$$



## 程式碼解釋


**注意**：僅用於解釋程式碼，這裡寫的是跑不起來的。


In [ ]:
# 想問 Notebook 常常 import 後其他的 cell 仍會有紅字，很醜看了不舒胡
import numpy as np
from typing import List
from config import settings, RuntimeState
from entities import Population

### 演算法參數設定

除錯小筆記：
在 Jupyter Notebook 中，程式碼儲存格是依序執行的，每個儲存格的執行結果會儲存在記憶體中。

如果你修改了一個儲存格的內容（例如修改了變數的值），但沒有重新執行與其相關的所有儲存格，那麼這些修改不會影響到已經執行過的程式碼。

In [ ]:
class GeneticAlgorithmConfig:
    POP_SIZE = 500  # 種群大小
    GENERATIONS = 100  # 世代數


class CrossoverConfig:
    CROSSOVER_TYPE = "uniform"  # "uniform", "blx_alpha", "linear_interpolation"
    BLX_ALPHA = 0.5  # 模糊交叉(Blend Crossover, BLX-α)


class MutationConfig:
    MUTATION_TYPE = "gaussian"  # "uniform", "gaussian"
    MUTATION_RATE = 0.6  # 變異率
    MUTATION_DELTA = 0.7  # 變異幅度

    GAUSSIAN_SIGMA = 0.5  # 高斯變異的標準差


class SelectionConfig:
    SELECTION_STRATEGY_TYPE = "rank_based"  # "roulette", "tournament", "rank_based"

    TOURNAMENT_SIZE = 5  # 錦標賽選擇(tournament)

### 目標函數

$$\text{Minimize } f(x, y) = -\cos(\pi y) - \exp(-\pi (x - 0.5)^2) \sin^2(\pi x)$$

In [ ]:
class ObjectiveFunction:
    @staticmethod
    def evaluate(x, y):
        return -np.cos(np.pi * y) - np.exp(-np.pi * (x - 0.5) ** 2) * np.sin(np.pi * x) ** 2


### 個體基因

兩個基因，分別代表 x, y，我使用 Real-valued Encoding 實數編碼。

   $$-1 \leq x \leq 2$$
   $$4 \leq y \leq 6$$
   $$x + y \geq 5$$

In [ ]:
class Individual:
    # 初始化個體，沒有提供 x 和 y 則隨機生成
    def __init__(self, x=None, y=None):
        if x is None or y is None:
            self.x = np.random.uniform(-1, 2)
            self.y = np.random.uniform(4, 6)
            while not Constraint.CONSTRAINT(self.x, self.y):
                # 重新生成直到滿足限制條件
                self.x = np.random.uniform(-1, 2)
                self.y = np.random.uniform(4, 6)


class Constraint:
    X_MIN = -1
    X_MAX = 2
    Y_MIN = 4
    Y_MAX = 6
    CONSTRAINT = lambda x, y: x + y >= 5


### 選擇方法

基因演算法（Genetic Algorithm, GA）中，選擇策略是決定哪些個體（解）會被選擇來產生下一代。這些策略的目的是確保好的個體有更高的機會被選中，同時維持一定的多樣性。

1. **基於排名的選擇（Rank-Based Selection）**：
    - **原理**：首先根據個體的適應度對整個種群進行排序，然後根據個體的排名來分配選擇概率。排名越高的個體選擇概率越大，但這種概率分配是基於排名而不是適應度。
    - **優點**：能夠避免適應度值差異過大導致的問題，保持種群多樣性，防止早熟收斂。
    - **缺點**：需要對種群進行排序，可能會增加計算複雜度。

2. **輪盤賭選擇（Roulette Wheel Selection）**：
    - **原理**：這種方法根據個體的適應度來選擇個體。適應度越高的個體被選中的概率越大。可以想像成一個輪盤，每個個體占據的區域大小與其適應度成正比。
    - **優點**：簡單直觀，適應度高的個體有較大機會被選中。
    - **缺點**：如果適應度差異很大，適應度低的個體幾乎沒有機會被選中，可能導致早熟收斂（premature convergence）。

3. **錦標賽選擇（Tournament Selection）**：
    - **原理**：從種群中隨機選擇一小部分個體（通常稱為錦標賽大小），然後從中選擇適應度最高的個體。這個過程重複多次，直到選出足夠多的個體。
    - **優點**：實現簡單，能夠調整選擇壓力（通過改變錦標賽大小來控制選擇壓力），適應度差異不大的情況下效果較好。
    - **缺點**：如果錦標賽大小過大，可能導致選擇壓力過大，從而導致多樣性降低；如果過小，選擇壓力過小，進化速度變慢。

總結：
- **Rank-Based Selection** 能夠有效避免適應度差異過大的問題，保持種群多樣性，但計算量相對較大。
- **Roulette Wheel Selection** 適合適應度分佈較均勻的情況，但可能會導致早熟收斂。
- **Tournament Selection** 靈活性較高，可以通過調整錦標賽大小來控制選擇壓力，適應範圍較廣。

In [ ]:
def rank_base_selection(population: List[Individual]) -> List[Individual]:
    fitness = np.array([individual.fitness() for individual in population])
    ranks = np.argsort(np.argsort(fitness))
    probabilities = ranks / ranks.sum()
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    return [population[i] for i in selected_indices]


def roulette_wheel_selection(population: List[Individual]) -> List[Individual]:
    fitness = np.array([individual.fitness() for individual in population])
    fitness -= fitness.min()
    if fitness.sum() == 0:
        fitness = np.ones_like(fitness)
    probabilities = fitness / fitness.sum()
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    return [population[i] for i in selected_indices]


def tournament_selection(population: List[Individual]) -> List[Individual]:
    fitness = np.array([individual.fitness() for individual in population])
    selected_indices = []
    for _ in range(len(population)):
        tournament = np.random.choice(len(population), size=SelectionConfig.TOURNAMENT_SIZE)
        best = tournament[np.argmax(fitness[tournament])]
        selected_indices.append(best)
    return [population[i] for i in selected_indices]


### 交配方法

1. **均勻交叉（Uniform Crossover）**：
    - **原理**：對於每個基因位置，隨機選擇來自哪個父代的基因。這種方法確保每個基因位置有50%的機會來自任一父代。
    - **優點**：提供了較高的基因多樣性，避免早熟收斂。
    - **缺點**：可能破壞優良基因組合。

2. **線性插值交叉（Linear Interpolation Crossover）**：
    - **原理**：通過線性插值的方法來生成子代基因。這種方法通過隨機生成一個係數 α，並根據 α 對兩個父代的基因進行加權平均。
    - **優點**：能夠生成父代基因之間的中間值，保留父代基因的優勢。
    - **缺點**：生成的子代可能會集中在父代之間的範圍內，可能會限制基因多樣性。

3. **BLX-α 交叉（Blend Crossover, BLX-α）**：
    - **原理**：這種方法通過在父代基因範圍的基礎上擴展一定的範圍來生成子代基因。 α 值決定了擴展範圍的大小。
    - **優點**：能夠生成比父代基因範圍更廣的子代，增加基因多樣性。
    - **缺點**：如果 α 值過大，可能會生成不合理的子代基因。

總結：
- **Uniform Crossove** 提供了高基因多樣性，但可能破壞優良基因組合。
- **Linear Interpolation Crossover** 保留了父代基因的優勢，但可能限制基因多樣性。
- **BLX-α** 提供了更廣的基因多樣性，但需要控制 α 值以避免生成不合理的基因。

In [ ]:
def uniform_crossover(parent1, parent2):
    x_child = parent1.x if np.random.rand() < 0.5 else parent2.x
    y_child = parent1.y if np.random.rand() < 0.5 else parent2.y
    return Individual(x_child, y_child)


def linear_interpolation_crossover(parent1, parent2):
    alpha = np.random.rand()
    x_child = alpha * parent1.x + (1 - alpha) * parent2.x
    y_child = alpha * parent1.y + (1 - alpha) * parent2.y
    return Individual(x_child, y_child)


def blx_alpha_crossover(parent1, parent2, alpha=CrossoverConfig.BLX_ALPHA):
    x_min = min(parent1.x, parent2.x)
    x_max = max(parent1.x, parent2.x)
    y_min = min(parent1.y, parent2.y)
    y_max = max(parent1.y, parent2.y)

    x_range = x_max - x_min
    y_range = y_max - y_min

    x_child = np.random.uniform(x_min - alpha * x_range, x_max + alpha * x_range)
    y_child = np.random.uniform(y_min - alpha * y_range, y_max + alpha * y_range)

    return Individual(x_child, y_child)


### 變異方法

1. **均勻變異（Uniform Mutation）**：
    - **原理**：以一定的機率對個體的基因進行變異，變異幅度在一個預定的範圍內隨機選取。
    - **優點**：簡單直觀，能夠有效引入基因多樣性。
    - **缺點**：變異幅度固定，可能導致變異效果不夠靈活。

2. **高斯變異（Gaussian Mutation）**：
    - **原理**：以一定的機率對個體的基因進行變異，變異幅度服從高斯分佈（正態分佈）。
    - **優點**：變異幅度服從高斯分佈，更加靈活，能夠適應不同的變異需求。
    - **缺點**：需要額外考慮高斯分佈的參數設定，可能增加實現的複雜度。

總結：
- **均勻變異** 提供了簡單的變異方法，適合變異幅度固定的情況。
- **高斯變異** 提供了更加靈活的變異方法，適合需要不同變異幅度的情況。

In [ ]:
def uniform_mutation(individual):
    if np.random.rand() < MutationConfig.MUTATION_RATE:  # 以 mutation_rate 的機率進行變異
        individual.x += np.random.uniform(-MutationConfig.MUTATION_DELTA, MutationConfig.MUTATION_DELTA)  # 變異幅度
        individual.y += np.random.uniform(-MutationConfig.MUTATION_DELTA, MutationConfig.MUTATION_DELTA)

        # 防止變異後超出範圍，限制在 Constraint 的範圍內
        individual.x = np.clip(individual.x, Constraint.X_MIN, Constraint.X_MAX)
        individual.y = np.clip(individual.y, Constraint.Y_MIN, Constraint.Y_MAX)

        # 如果變異後不符合限制條件，
        if not Constraint.CONSTRAINT(individual.x, individual.y):
            # 隨機調整 x 或 y 以滿足約束條件
            if np.random.rand() < 0.5:
                individual.x = 5 - individual.y
                individual.x = np.clip(individual.x, Constraint.X_MIN, Constraint.X_MAX)  # 再次確認 x 在範圍內
            else:
                individual.y = 5 - individual.x
                individual.y = np.clip(individual.y, Constraint.Y_MIN, Constraint.Y_MAX)  # 再次確認 y 在範圍內

    return individual.x, individual.y


def gaussian_mutation(individual, sigma=MutationConfig.GAUSSIAN_SIGMA):
    if np.random.rand() < MutationConfig.MUTATION_RATE:
        individual.x += np.random.normal(0, sigma)
        individual.x = np.clip(individual.x, Constraint.X_MIN, Constraint.X_MAX)  # 確保 x 在範圍內
    if np.random.rand() < MutationConfig.MUTATION_RATE:
        individual.y += np.random.normal(0, sigma)
        individual.y = np.clip(individual.y, Constraint.Y_MIN, Constraint.Y_MAX)  # 確保 y 在範圍內
    # 確保 x 和 y 的約束條件
    while not Constraint.CONSTRAINT(individual.x, individual.y):
        individual.x = np.random.uniform(Constraint.X_MIN, Constraint.X_MAX)
        individual.y = np.random.uniform(Constraint.Y_MIN, Constraint.Y_MAX)

    return individual.x, individual.y


### 整體流程

1. **初始化**：隨機生成一個初始種群。
2. **適應度計算**：計算每個個體的適應度。
3. **選擇**：根據適應度選擇父代個體。
4. **交配**：根據交配機率對父代進行交配，生成子代。
5. **變異**：根據變異機率對子代進行變異。
6. **更新**：根據適應度選擇子代和父代中的較好個體，生成下一代種群。
7. **終止條件**：達到最大迭代次數。

In [ ]:
class GeneticAlgorithm:

    def __init__(self):
        self.GAconfig = settings.GeneticAlgorithmConfig()
        self.individual = Individual()
        self.population = Population()
        self.runtime_stats = RuntimeState()

    def run(self):
        # 初始化全局最佳個體和適應度
        global_best_individual = self.population.get_best_individual()
        self.runtime_stats.set_global_best_individual(global_best_individual)
        self.runtime_stats.set_global_best_fitness(global_best_individual.fitness())

        # 開始迭代
        for generation in range(self.GAconfig.get_generations()):
            # 選擇父母
            parents = self.population.select_parents()
            # 生成下一代
            self.population.generate_next_population(parents)
            # 計算當前最佳個體
            best_individual = self.population.get_best_individual()

            # 更新全局最佳個體和適應度
            if best_individual.fitness() < self.runtime_stats.get_global_best_fitness():
                self.runtime_stats.set_global_best_individual(best_individual)
                self.runtime_stats.set_global_best_fitness(best_individual.fitness())
            # 輸出當前世代的統計資料
            self.__log_generation_stats(best_individual)
            # 更新計數
            self.runtime_stats.increment_current_generation()

        # 輸出最終結果
        self.__print_final_result()


## 檢視結果：Log 紀錄

### 此次紀錄設定檔

In [ ]:
class GeneticAlgorithmConfig:
    POP_SIZE = 5000  # 種群大小
    GENERATIONS = 100  # 世代數


class CrossoverConfig:
    CROSSOVER_TYPE = "uniform"


class MutationConfig:
    MUTATION_TYPE = "gaussian"
    MUTATION_RATE = 0.6
    MUTATION_DELTA = 0.7

    GAUSSIAN_SIGMA = 0.5


class SelectionConfig:
    SELECTION_STRATEGY_TYPE = "rank_based"


### 仍存在問題

1. 我的演算法常常交配後結果越來越差。
2. `x` 無法收斂到整數 0.5000000，但是 `y` 就可以，可能是`Individual`寫法有問題，不知道。
3. 感覺效率很差，需要很大的族群數才能有好結果。